In [2]:
#ライブラリのインポート
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import time
import datetime as dt
from matplotlib import pyplot as plt

import PySimpleGUI as sg

import plotly
import plotly.graph_objs as go
# Google Colab. やJupyter Lab.でプロットするためには，以下を実行する．
import plotly.io as pio
#グラフをブラウザ表示する場合に有効化
pio.renderers.default='browser'
#pio.renderers.default = "colab"

In [3]:
#変数の定義
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

#記録イベントのためにstartedという関数を定義               
started= False

#角度・時間のリストに角度を追記していく
#変数の初期化
mp_joint_anglelist=[]
ip_joint_anglelist=[]
tip_joint_anglelist=[]
timelist=[]
inputname_recordtime=''
mp_distancelist = []
tip_distancelist=[]
palmsize1_list=[]
palmsize2_list=[]

true_anglelist=[]

In [4]:
#各種関数の定義
def get_label(index, hand, results):#左右どちらかの手を判定する関数の定義
    output = None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index==index:
            
            #Process results
            label=classification.classification[0].label
            score=classification.classification[0].score
            text ='{} {}'.format(label,round(score,2))
            
            #Extraxt Coordinates
            coords = tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x,hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                                     [640,480]).astype(int))
            output = text, coords
    return output
#3次元座標の角度計算の定義
def calculate_3D_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #0がx座標　１がｙ座標　２がｚ座標
    #ベクトルを定義 長さと内積からコサインを計算
    vec_a=a-b
    vec_c=c-b
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_c=np.linalg.norm(vec_c)
    inner_product=np.inner(vec_a, vec_c)
    cos=inner_product/(len_vec_a*len_vec_c)
    
    #角度（ラジアン）の計算
    rad=np.arccos(cos)
    
    #ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle
#角度計算の関数の定義
def draw_finger_angles(image, results, joint_list):
    # Loop through hands
    for hand in results.multi_hand_landmarks:
        #Loop through joint sets 
        for joint in joint_list:
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) # First coord
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) # Second coord
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) # Third coord
            
            radians = np.arctan2(c[1] - b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
            angle = np.abs(radians*180.0/np.pi)
            
            if angle > 180.0:
                angle = 360-angle
            #cv.putText関数をチェックしておくこと    
            cv2.putText(image, str(round(angle, 1)), tuple(np.multiply(c, [640, 480]).astype(int)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    return image, angle

#4点（２つのベクトル）からの角度計算の関数の定義
def calculate_2vec_angle(a,b,c,d):
    a = np.array(a) # X1
    b = np.array(b) # X2
    c = np.array(c) # X3
    d = np.array(d) # X4
  #[x,y,z]座標から[x,y]のみスライスしてベクトルを作成
    vec_a=a[0:2]-b[0:2]
    vec_c=c[0:2]-d[0:2]
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_c=np.linalg.norm(vec_c)
    inner_product=np.inner(vec_a, vec_c)
    cos=inner_product/(len_vec_a*len_vec_c)
#角度（ラジアン）の計算
    rad=np.arccos(cos)
#ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

#2次元からの角度計算の定義
def calculate_2D_angle(a,b,c):
    a=np.array(a)#firtst
    b=np.array(b)#second
    c=np.array(c)#third
  #[x,y,z]座標から[x,y]のみスライスしてベクトルを作成
    vec_a=a[0:2]-b[0:2]
    vec_c=c[0:2]-b[0:2]
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_c=np.linalg.norm(vec_c)
    inner_product=np.inner(vec_a, vec_c)
    cos=inner_product/(len_vec_a*len_vec_c)
#角度（ラジアン）の計算
    rad=np.arccos(cos)
#ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

#指間の距離計測の関数 a-b（母指の示指の座標）間の距離を手掌の大きさ(c-dの距離）で除したものとする
def point_distance(a,b,c,d):
    a=np.array(a)#母指の座標
    b=np.array(b)#示指の座標
    c=np.array(c)#遠位の座標
    d=np.array(d)#近位の座標
    
    fing_dist=np.linalg.norm(a-b)
    palm_size=np.linalg.norm(c-d)
    distance=fing_dist/palm_size
    return distance 

#手掌面積　palmsizeの計算 2Dの3点から外積を算出　
def calc_size(a, b, c):
    a=np.array(a)#橈側先端の座標
    b=np.array(b)#尺側先端の座標
    c=np.array(c)#基部（手関節）の座標

    rad_palm=a-c
    uln_palm=b-c
    
    size1=np.cross(rad_palm,uln_palm)*10#数字が小さいので10倍しておく
    return size1

In [5]:
#plotlyでグラフを可視化の関数定義
#combine Anglelist arrayにして水平結合
#timelistの開始数字を0に変更
#np.array(timelist[0])
#np.full(len(timelist),timelist[0])#すべての要素がtimelistの１番目の値でlen(timelist)の行列を作成
#経過時間 deltatimeを計算（1番目の要素を0として行列の引き算)
def draw_graph(timelist, mp_joint_anglelist, ip_joint_anglelist, tip_joint_anglelist, mp_distancelist, tip_distancelist, palmsize1_list, palmsize2_list):
    if len(timelist) == 0:
        pass
    else:
        """timelist=timelist[1:]
    mp_joint_anglelist=mp_joint_anglelist[1:]
    ip_joint_anglelist=ip_joint_anglelist[1:]
    tip_joint_anglelist=tip_joint_anglelist[1:]"""
    
        deltatime=np.array(timelist)-np.full(len(timelist),timelist[0])
        mp_angle=np.array(mp_joint_anglelist)
        ip_angle=np.array(ip_joint_anglelist)
        tip_angle=np.array(tip_joint_anglelist)
        mp_distance=np.array(mp_distancelist)
        tip_distance=np.array(tip_distancelist)
        palmsize1=np.array(palmsize1_list)
        palmsize2=np.array(palmsize2_list)

#plotlyによるデータの可視化
    fig = go.Figure() # 1 

    fig.add_trace(    #2. グラフを追加・記述する
        go.Scatter(x = deltatime, y = mp_angle,
                  name='mp angle')
        )
    fig.add_trace(
        go.Scatter(x= deltatime, y =ip_angle, name='ip angle')
        )
    fig.add_trace(
        go.Scatter(x=deltatime, y= tip_angle, name='tip angle')
        )
    fig.add_trace(
        go.Scatter(x=deltatime, y= mp_distance, name='mp_distance')
        )
    fig.add_trace(
        go.Scatter(x=deltatime, y= tip_distance, name='tip distance')
        )
    fig.add_trace(
        go.Scatter(x=deltatime, y= palmsize1, name='palmsize1')
        )
    fig.add_trace(
        go.Scatter(x=deltatime, y= palmsize2, name='paimsize2')
        )

    fig.update_layout( #3. グラフの装飾、制限を追加(titleの表示、x軸の最大値など)
        title = 'Joint angles and distance over time',
        xaxis_title = 'time',
        yaxis_title = 'angle or distance',
        showlegend = True
        )
    fig.show() #4.グラフの描画
    return

In [42]:
#初期化する関数宣言した変数をグローバス宣言してからリストを初期化
def clear_log():
    global timelist, mp_joint_anglelist, ip_joint_anglelist, tip_joint_anglelist, mp_distancelist, tip_distancelist,palmsize1_list,palmsize2_list
    mp_joint_anglelist=[]
    ip_joint_anglelist=[]
    tip_joint_anglelist=[]
    timelist=[]
    mp_distancelist=[]
    tip_distancelist=[]
    palmsize1_list=[]
    palmsize2_list=[]
    return 

#計測データをCSV保存する関数
#角度データをAngle＿Results.CSVとして保存
def save_csv():
    name=['time','mp_joint_angle', 'ip_joint_angle', 'tip_joint_angle','mp_distancelist', 'tip_distancelist','palmsize1','palmsize2','trueangle']
    deltatime=np.array(timelist)-np.full(len(timelist),timelist[0])
    mp_angle=np.array(mp_joint_anglelist)
    ip_angle=np.array(ip_joint_anglelist)
    tip_angle=np.array(tip_joint_anglelist)
    mp_distance=np.array(mp_distancelist)
    tip_distance=np.array(tip_distancelist)    
    palmsize1=np.array(palmsize1_list)
    palmsize2=np.array(palmsize2_list)
    trueangle=np.array(true_anglelist)

    data=np.stack((deltatime,mp_angle,ip_angle,tip_angle,mp_distance,tip_distance,palmsize1,palmsize2,trueangle)).T
    data=np.vstack((name,data))
    #global now
    now = dt.datetime.now()
    #global recordtime
    recordtime = now.strftime('%Y%m%d-%H%M%S')
    global inputname_recordtime
    inputname_recordtime=str(basename+'_'+recordtime)
    np.savetxt('result_{}.csv'.format(inputname_recordtime),data,delimiter = ',',fmt = '%s')
    return


In [7]:
#GUIのレイアウト設定

sg.theme('SystemDefault')
layout = [
   [sg.Image(key='img1',), sg.Image(key='img2',)],
    [sg.Text('Input Name:'), sg.Text(size=(15,1), key='input_name')],
    [sg.Input(key='-IN-')],

    [[sg.Button('Start Log', size=(10, 1)), sg.Button('Stop Log', size=(10,1))]],
    [[sg.Button('Show Graph',size=(10,1)),sg.Button('Save CSV',size=(10,1)),sg.Button('Clear Log',size=(10,1))]],
    [sg.Button('Exit', size=(10, 1))],]



In [8]:
#true_angle = sg.popup_get_text('input ture angle', 'Input TrueAngle')

#sg.popup('入力結果', 'True Angle is ', text)

In [30]:
filename = sg.popup_get_file('ファイルを入力する')
sg.popup('入力完了！', filename)
basename=os.path.basename(filename)
print(f"basename: {os.path.basename(filename)}")


basename: 1-40.mp4


In [54]:
#GUIでファイルを指定してファイル名を取得
filename = sg.popup_get_file('ファイルを入力する')
sg.popup('入力完了！', filename)
basename=os.path.basename(filename)
print(f"basename: {os.path.basename(filename)}")

#角度の真値をpopup入力
true_angle = sg.popup_get_text('input ture angle', 'Input TrueAngle')

# webカメラをキャプチャー
cap = cv2.VideoCapture(filename)
# ビデオ記録用の変数定義
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('result_{}.mp4'.format(dt.datetime.now()), fourcc, fps, (w, h))
#pysimpleGUIのwindow定義
#window = sg.Window("webカメラ画面", location=(100,100),layout=layout, size=(1600,1600),resizable=True,finalize=True)
# イベントループ
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # 取得に失敗した場合

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        #imagesizeを小さくする
        image = cv2.resize(image, (round(0.6*w), round(0.6*h)))
        
        # Detections
        #print(results)
        
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                   )
         #中指MPと手関節中心を基準軸に外転角度計算          
            a=hand.landmark[9].x, hand.landmark[9].y
            b=hand.landmark[0].x, hand.landmark[0].y
            c=hand.landmark[2].x, hand.landmark[2].y
            mp_joint_angle=calculate_2D_angle(a,b,c)
            
        # ip_joint_angle(2Dで)
            a1=hand.landmark[9].x, hand.landmark[9].y
            b1=hand.landmark[0].x, hand.landmark[0].y
            c1=hand.landmark[3].x, hand.landmark[3].y
            ip_joint_angle=calculate_2D_angle(a1,b1,c1)
            
        # tip_joint_angle(2Dで)
            a2=hand.landmark[9].x, hand.landmark[9].y
            b2=hand.landmark[0].x, hand.landmark[0].y
            c2=hand.landmark[4].x, hand.landmark[4].y
            tip_joint_angle=calculate_2D_angle(a2,b2,c2)
        # mp_distance(2D)で
            a3=hand.landmark[2].x, hand.landmark[2].y#母指MP
            b3=hand.landmark[5].x, hand.landmark[5].y#示指MP
            c3=hand.landmark[0].x, hand.landmark[0].y#手関節
            d3=hand.landmark[5].x, hand.landmark[5].y#示指MP
            mp_distance=point_distance(a3,b3,c3,d3)
        # tip_distance(2D)で
            a4=hand.landmark[4].x, hand.landmark[4].y#母指tip
            b4=hand.landmark[8].x, hand.landmark[8].y#示指tip
            c4=hand.landmark[0].x, hand.landmark[0].y#手関節
            d4=hand.landmark[5].x, hand.landmark[5].y#示指MP
            tip_distance=point_distance(a4,b4,c4,d4)
        #palmsize1の計算
            a5=hand.landmark[5].x, hand.landmark[5].y#示指MP
            b5=hand.landmark[17].x, hand.landmark[17].y#小指MP
            c5=hand.landmark[0].x, hand.landmark[0].y#手関節
            result_calc_size1=calc_size(a5, b5, c5)
            palmsize1=abs(result_calc_size1)
        #palmsize2の計算
            a6=hand.landmark[3].x, hand.landmark[3].y#母指IP
            b6=hand.landmark[6].x, hand.landmark[6].y#示指PIP
            c6=hand.landmark[0].x, hand.landmark[0].y#手関節
            result_calc_size2=calc_size(a6, b6, c6)
            palmsize2=abs(result_calc_size2)

        
        

    # キャプチャーした画像をpngに変換
           #image1 = cv2.imencode('.png', image)[1].tobytes()

    # Imageの内容を更新
        #window['img1'].update(data=image1)
                    
        #cv2.putText(image, text, coord, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # Draw angles to image from joint list
            #draw_finger_angles(image, results, joint_list)
       
        #cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        #cv2.imshow('Hand Tracking', image)

        """event, values = window.read(timeout=0)
        window['input_name'].update(values['-IN-'])
        
        if event == 'Start Log' and results.multi_hand_landmarks:
            started = True
            
        if started == True:"""
        #角度情報をテキストして画像に挿入
        cv2.putText(image, 'MP_angle'+str(round(mp_joint_angle,1)),
                                  (50,450),
              #tuple(np.multiply(rtelbow[:2],[w, h]).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,150,150),2,cv2.LINE_AA)

        cv2.putText(image, 'IP_angle'+str(round(ip_joint_angle,1)),
                                  (50,400),
              #tuple(np.multiply(rtshoulder[:2],[w, h]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,150,150),2,cv2.LINE_AA)
        cv2.putText(image, 'tip_angle'+str(round(tip_joint_angle,1)),
                                  (50,350),
              #tuple(np.multiply(rtshoulder[:2],[w, h]).astype(int)),
                                  cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,150,150),2,cv2.LINE_AA)                 
        #距離情報をテキストして画像に挿入
        cv2.putText(image, 'MP_distance'+str(round(mp_distance,1)),
                                  (300,400),
              #tuple(np.multiply(rtelbow[:2],[w, h]).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,150,150),2,cv2.LINE_AA)

        cv2.putText(image, 'TIP_distance'+str(round(tip_distance,1)),
                                  (300,450),
              #tuple(np.multiply(rtshoulder[:2],[w, h]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,150,150),2,cv2.LINE_AA)
        #面積情報をテキストして画像に挿入
        cv2.putText(image, 'PalmSize1_'+str(round(palmsize1,3)),
                                  (500,400),
              #tuple(np.multiply(rtelbow[:2],[w, h]).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,150,150),2,cv2.LINE_AA)

        cv2.putText(image, 'PalmSize2_'+str(round(palmsize2,3)),
                                  (500,450),
              #tuple(np.multiply(rtshoulder[:2],[w, h]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,150,150),2,cv2.LINE_AA)
            
        #結果の可視化
        cv2.imshow("mediapipe_image", image)   
        timelist.append(time.time())
        mp_joint_anglelist.append(mp_joint_angle)
        ip_joint_anglelist.append(ip_joint_angle)
        tip_joint_anglelist.append(tip_joint_angle)
        mp_distancelist.append(mp_distance)
        tip_distancelist.append(tip_distance)

        palmsize1_list.append(palmsize1)
        palmsize2_list.append(palmsize2)

        true_anglelist.append(true_angle)
    #画像を録画用に保存
        """out.write(image)                   
    # キャプチャーした画像をpngに変換
            image2 = cv2.imencode('.png', image)[1].tobytes()
    # Imageの内容を更新
            window['img2'].update(data=image2)       

        if event =='Stop Log':
            started = False
            out.release()

        if event == 'Show Graph':
            draw_graph(timelist, mp_joint_anglelist, ip_joint_anglelist, tip_joint_anglelist,mp_distancelist,tip_distancelist,palmsize1_list,palmsize2_list)
        if event == 'Clear Log':
            clear_log()
        if event == 'Save CSV':
            save_csv()"""

    # qを押せば処理を中止できるようにしておく
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"): break
    cap.release()
    cv2.destroyAllWindows()
    
    save_csv()
    print('Saved data is '+inputname_recordtime)
    print('Data length is '+str(len(mp_joint_anglelist)))

basename: 4-60.mp4
Saved data is 4-60.mp4_20221101-112913
Data length is 4299
